# Google Cloud Vision Dossard Recognition

## Prise en main 

### Mise en place de l'authentification API de Google Cloud

La clé API a été déclarée en tant que telle suivant la documentation ici: https://cloud.google.com/docs/authentication/getting-started?hl=fr#auth-cloud-implicit-python

On teste si la connexion est bien acceptée en faisant un test sur une image en local.
Je n'ai pas réussi à utiliser une variable globale GOOGLE_APPLICATION_CREDENTIALS sur mon Unix, l'Auth n'arrive pas à la récupérer. (me dit que le fichier est inexistant). Je définis donc la variable après mes imports.

In [ ]:
import io
import os

# Imports the Google Cloud client library
from google.cloud import vision
from google.cloud.vision import types


# Instantiates a client
client = vision.ImageAnnotatorClient()

# The name of the image file to annotate
file_name = os.path.join(
    os.path.dirname("__file__"),
    'resources/2.png')

# Loads the image into memory
with io.open(file_name, 'rb') as image_file:
    content = image_file.read()

image = types.Image(content=content)

# Performs label detection on the image file
response = client.label_detection(image=image)
labels = response.label_annotations

print('Labels:')
for label in labels:
    print(label.description)

Labels:
path
tree
woody plant
trail
mountain bike
woodland
duathlon
ultramarathon
bicycle racing
bicycle


### Hello Vision

On fait le même test avec une autre rédaction, sous forme de call JSON. On va aussi chercher l'image sur un bucket google plutôt que en local.

In [2]:
client2 = vision.ImageAnnotatorClient()
response2 = client2.annotate_image({'image': {'source': {'image_uri': 'gs://bib_sample/2.png'}},'features': [{'type': vision.enums.Feature.Type.LABEL_DETECTION}],})
labels = response2.label_annotations
print('Labels:')
for label in labels:
    print(label.description)

Labels:
path
tree
woody plant
trail
mountain bike
woodland
duathlon
ultramarathon
bicycle racing
bicycle


### Hello Text Recognition

#### 1er test : reconnaissance de texte sur image full size

On essaie maintenant de faire un call pour repérer du texte. Voici l'image sur laquelle on travaille:

<img src="resources/2.png" width="400" height="400" />

In [3]:
client3 = vision.ImageAnnotatorClient()
response3 = client3.annotate_image({'image': {'source': {'image_uri': 'gs://bib_sample/2.png'}},'features': [{'type': vision.enums.Feature.Type.TEXT_DETECTION}],})
texts = response3.text_annotations
print('Textes:')
for text in texts:
    print('\n"{}"'.format(text.description))

Textes:


Le code ne renvoie rien, bizarre...On voudrait qu'il y ait 464. On testera donc ensuite sur un zoom sur les dossards.

#### 2ème test : reconnaissance de texte sur image zoomée

<img src="resources/2-zoom.png" width="400" height="400" />

In [4]:
client4 = vision.ImageAnnotatorClient()
response4 = client4.annotate_image({'image': {'source': {'image_uri': 'gs://bib_sample/2-zoom.png'}},'features': [{'type': vision.enums.Feature.Type.TEXT_DETECTION}],})
texts = response4.text_annotations
print('Textes:')
for text in texts:
        print('\n"{}"'.format(text.description))

        vertices = (['({},{})'.format(vertex.x, vertex.y)
                    for vertex in text.bounding_poly.vertices])

        print('bounds: {}'.format(','.join(vertices)))

Textes:

"464
"
bounds: (166,113),(240,113),(240,150),(166,150)

"464"
bounds: (167,113),(240,116),(238,150),(166,147)


L'IA a bien repéré 464! Par contre le dossard a été repérée 2 fois, a des positions très semblables. 
Soit au moins 2 points à cruncher: 
- trouver un dossard avant d'essayer de le lire.
- vérifier les doublons

#### 3ème test: reconnaissance sur le crop hint de google

Une feature de crop hint existe dans l'API. Elle tend à renvoyer la partie importante d'ume image. On va réaliser un crop hint de l'image full size, puis réaliser une détection de texte sur l'image cropé ainsi. On défini les fonctions de crop à l'aide de la documentation donnée ici : https://cloud.google.com/vision/docs/crop-hints?hl=fr

In [5]:
import argparse

from PIL import Image, ImageDraw

def get_crop_hint(path):
    """Detect crop hints on a single image and return the first result."""
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = types.Image(content=content)

    crop_hints_params = types.CropHintsParams(aspect_ratios=[1.77])
    image_context = types.ImageContext(crop_hints_params=crop_hints_params)

    response = client.crop_hints(image=image, image_context=image_context)
    hints = response.crop_hints_annotation.crop_hints

    # Get bounds for the first crop hint using an aspect ratio of 1.77.
    vertices = hints[0].bounding_poly.vertices

    return vertices

def get_crop_hint_Bucket(bucketPath):
    """Detect crop hints on a single image and return the first result."""
    client = vision.ImageAnnotatorClient()
    
    source = types.ImageSource(bucketPath)

    image = types.Image(source=source)

    crop_hints_params = types.CropHintsParams(aspect_ratios=[1.77])
    image_context = types.ImageContext(crop_hints_params=crop_hints_params)

    response = client.crop_hints(image=image, image_context=image_context)
    hints = response.crop_hints_annotation.crop_hints

    # Get bounds for the first crop hint using an aspect ratio of 1.77.
    vertices = hints[0].bounding_poly.vertices

    return vertices

def draw_hint(image_file):
    """Draw a border around the image using the hints in the vector list."""
    vects = get_crop_hint(image_file)

    im = Image.open(image_file)
    draw = ImageDraw.Draw(im)
    draw.polygon([
        vects[0].x, vects[0].y,
        vects[1].x, vects[1].y,
        vects[2].x, vects[2].y,
        vects[3].x, vects[3].y], None, 'red')
    im.save('output-hint.jpg', 'JPEG')
    
def crop_to_hint(image_file):
    """Crop the image using the hints in the vector list."""
    vects = get_crop_hint(image_file)

    im = Image.open(image_file)
    im2 = im.crop([vects[0].x, vects[0].y,
                  vects[2].x - 1, vects[2].y - 1])
    im2.save('output-crop.jpg', 'JPEG')
    
def draw_hint_PNG(image_file):
    """Draw a border around the image using the hints in the vector list."""
    vects = get_crop_hint(image_file)

    im = Image.open(image_file)
    draw = ImageDraw.Draw(im)
    draw.polygon([
        vects[0].x, vects[0].y,
        vects[1].x, vects[1].y,
        vects[2].x, vects[2].y,
        vects[3].x, vects[3].y], None, 'red')
    im.save('output-hint4.png', 'PNG')
    
def crop_to_hint_PNG(image_file):
    """Crop the image using the hints in the vector list."""
    vects = get_crop_hint(image_file)

    im = Image.open(image_file)
    im2 = im.crop([vects[0].x, vects[0].y,
                  vects[2].x - 1, vects[2].y - 1])
    im2.save('output-crop4.png', 'PNG')

    
##Parser de fonction main / call Pas utile pour le moment    
#parser = argparse.ArgumentParser()
#parser.add_argument('image_file', help='The image you\'d like to crop.')
#parser.add_argument('mode', help='Set to "crop" or "draw".')
#args = parser.parse_args()

#parser = argparse.ArgumentParser()

#if args.mode == 'crop':
#    crop_to_hint(args.image_file)
#elif args.mode == 'draw':
#    draw_hint(args.image_file)

Les fonctions sont bien définies. On teste sur une image: 

In [6]:
# fonction pour afficher un cadre autour du hint: 
draw_hint_PNG('resources/4.png')

# fonction pour créer un crop
crop_to_hint_PNG('resources/4.png')
print('done')

done


Les résultats sont plutôt médiocres sur les images testées: 
<img src="crop-hint-output/output-hint4.png" width="400" height="400" /><img src="crop-hint-output/output-hint1.png" width="400" height="400" /><img src="crop-hint-output/output-hint2.png" width="400" height="400" />

On va donc devoir essayer de répérer les personnes plutot qu'utiliser le hint. On sait qu'une personne doit porter un dossard. Il faudra essayer de faire le focus sur le corps, et si possible sur le buste, et ne pas se contenter de la tête comme semblent le faire les calls basiques de l'API Vision.

## Détection de corps

## Call sur une partie d'une image

Suite à la détection des corps, ou de la zone ou des corps sont présents, on peut faire un call sur cette zone de l'image. On va comparer la detection de texte sur une partie d'image et sur une image complète.

In [16]:
#Detection sur image entière:
client10 = vision.ImageAnnotatorClient()
response10 = client10.annotate_image({'image': {'source': {'image_uri': 'gs://bib_sample/2.png'}},'features': [{'type': vision.enums.Feature.Type.TEXT_DETECTION}],})
texts = response10.text_annotations
print('Textes:')
for text in texts:
    print('\n"{}"'.format(text.description))
print('Fin des Textes')

Textes:
done


La recherche de texte sur l'image 2 complète ne renvoie rien. On essaie maintenant sur une partie de l'image:

In [18]:
#Detection sur partie de l'image:
client11 = vision.ImageAnnotatorClient()
response11 = client11.annotate_image({"requests": [{"image": {"source": {"imageUri": "gs://bib_sample/2.png"}},"features": [{"type": "TEXT_DETECTION"}],"imageContext": {"latLongRect": {"maxLatLng": {"latitude": 50,"longitude": 0},"minLatLng": {"latitude": -70,"longitude": -170}}}}]})
texts = response11.text_annotations
print('Textes:')
for text in texts:
    print('\n"{}"'.format(text.description))
print('Fin des Textes')

KeyError: 'image'

## Traitement d'une liste d'image

Sujet traité par Pop's.

## Remarques

Il est tout à fait possible de laisser les image en privé dans le bucket.